In [73]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose
import nibabel as nib
import numpy as np
from tqdm import tqdm
from torchvision.transforms import Compose

In [74]:
# Custom dataset class for handling NIfTI files
class BrainDataset(Dataset):
    def __init__(self, t1w_files, t2w_files, fa_files, adc_files, transform=None):
        self.t1w_files = t1w_files
        self.t2w_files = t2w_files
        self.fa_files = fa_files
        # self.adc_files = adc_files
        self.transform = transform

    def __len__(self):
        return len(self.t1w_files)

    def __getitem__(self, idx):
        t1w_image = nib.load(self.t1w_files[idx]).get_fdata()
        t2w_image = nib.load(self.t2w_files[idx]).get_fdata()
        fa_image = nib.load(self.fa_files[idx]).get_fdata()
        # adc_image = nib.load(self.adc_files[idx]).get_fdata()

        # input_image = np.stack([t1w_image, t2w_image], axis=0)
        t1w_image = np.stack([t1w_image], axis=0)
        # target_image = np.stack([fa_image, adc_image], axis=0)
        t2w_image = np.stack([t2w_image], axis=0)
        fa_image = np.stack([fa_image], axis=0)

        # if self.transform:
            # input_image = self.transform(input_image)
            # target_image = self.transform(target_image)

        # return input_image, target_image
        return t1w_image, t2w_image, fa_image

In [75]:
cd ../..

c:\Users\keert\project2


In [76]:
os.chdir('./data/input')

patient_folders = [folder for folder in os.listdir() if os.path.isdir(folder) and folder.startswith('0')]

t1w_files = []
t2w_files = []
fa_files = []
adc_files = []

for patient_folder in patient_folders:
    path = os.path.join(patient_folder, patient_folder,)

    t1w_files.append(os.path.join(path, "T1w_1mm.nii.gz"))
    t2w_files.append(os.path.join(path, "T2w_1mm_noalign.nii.gz"))
    adc_files.append(os.path.join(path, "ADC_deformed.nii.gz"))
    fa_files.append(os.path.join(path, "FA_deformed.nii.gz"))

    # registered_path = os.path.join(patient_folder, patient_folder, 'registered')
    # normalized_path = os.path.join(patient_folder, patient_folder, 'normalized')

    # t1w_files.append(os.path.join(normalized_path, "T1w_1mm_normalized.nii.gz"))
    # t2w_files.append(os.path.join(registered_path, "T2w_registered.nii.gz"))
    # adc_files.append(os.path.join(registered_path, "ADC_registered.nii.gz"))
    # fa_files.append(os.path.join(registered_path, "FA_registered.nii.gz"))

# dataset = BrainDataset(t1w_files, t2w_files, fa_files, adc_files, transform=Compose([torch.tensor]))
dataset = BrainDataset(fa_files, adc_files, fa_files, adc_files, transform=Compose([torch.tensor]))
dataloader = DataLoader(dataset, batch_size=2, shuffle=True) #, num_workers=2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 0 for FA, 1 for ADC
output_modality = 0

print(t1w_files)

['001\\001\\T1w_1mm.nii.gz', '002\\002\\T1w_1mm.nii.gz', '003\\003\\T1w_1mm.nii.gz', '004\\004\\T1w_1mm.nii.gz']


In [77]:
# for inputs, targets in tqdm(dataloader):
#     inputs, targets = inputs.to(device), targets.to(device)

#     print(inputs.shape)
#     print(targets.shape)

for t1, t2, fa in tqdm(dataloader):
    t1, t2, fa = t1.to(device), t2.to(device), fa.to(device)

    print(t1.shape)
    print(t2.shape)

 50%|█████     | 1/2 [00:00<00:00,  1.38it/s]

torch.Size([2, 1, 145, 174, 145])
torch.Size([2, 1, 145, 174, 145])


100%|██████████| 2/2 [00:01<00:00,  1.43it/s]

torch.Size([2, 1, 145, 174, 145])
torch.Size([2, 1, 145, 174, 145])


In [78]:
input_shape = (1, 145, 174, 145)

nc = 1 # num channels

ngf = 32 # size of feature maps in generator

ndf = 32 # size of feature maps in discriminator

num_epochs = 1 # 200

lr = 0.0002

betas = (0.5, 0.999) # beta1 hyperparameter for Adam optimizers

ngpu = 1 # number of GPUs available, 0 for CPU mode

batch_size = 128 # batch size during training

latent_dim = 100

ngpu = 0 # Number of GPUs available. Use 0 for CPU mode.

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [79]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.convnet = nn.Sequential(
            # input is 1 x 145 x 174 x 145
            nn.Conv3d(1, 32, kernel_size=(3, 3, 3), padding=1),
            nn.MaxPool3d(kernel_size=(2, 2, 2)),
            # input is 32 x 72 x 87 x 72
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=1),
            nn.MaxPool3d(kernel_size=(2, 2, 2)),
            # input is 64 x 36 x 43 x 36
            nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=1),
            nn.MaxPool3d(kernel_size=(2, 2, 2)),
            nn.Flatten(),
            # input is 128 x 18 x 21 x 18
            nn.Linear(128 * 18 * 21 * 18, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.LeakyReLU()
            # nn.Linear(128, 64),
            # nn.Linear(64, 1),
            # nn.ReLU(),
            # nn.Sigmoid(), # is this needed?
        )

    def forward(self, x):
        return self.convnet(x)

In [80]:
t1_model = ConvNet()
t2_model = ConvNet()

In [82]:
# Define the input shape
img_shape = (2, 145, 174, 145)

# Define the generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc_layer = nn.Sequential(
            nn.Linear(latent_dim * 2, 128 * 18 * 21 * 18),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d( 128 * 18 * 21 * 18)
        )
        self.conv_layers = nn.Sequential(
            nn.ConvTranspose3d(128, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), output_padding=(1, 1, 1)),
            nn.BatchNorm3d(64),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose3d(64, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), output_padding=(1, 1, 1)),
            nn.BatchNorm3d(32),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose3d(32, 1, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), output_padding=(1, 1, 1)),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.fc_layer(x)
        x = x.view(x.shape[0], 128, 18, 21, 18) # reshaping tensor
        x = self.conv_layers(x)
        return x

In [84]:
# Define the discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1)),
            nn.LeakyReLU(0.2),
            nn.Dropout3d(0.25),
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1)),
            nn.LeakyReLU(0.2),
            #nn.Dropout3d(0.25),
            nn.Flatten(),
            # nn.Linear(64 * 23* 28 * 23, 128),
            # nn.LeakyReLU(0.2), #inplace=True
            # nn.Linear(128, 64),
            # nn.LeakyReLU(0.2), #inplace=True
            # nn.Linear(64, 1),
            # nn.Linear(64 * 36 * 43 * 36, 1),
            nn.Linear(3855104, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.main(x)

In [85]:
# Create the discriminator and generator models
discriminator = Discriminator()
generator = Generator()

# generator.apply(weights_init)
# discriminator.apply(weights_init)

# Define the loss function and optimizer for the discriminator and generator
adversarial_loss = nn.BCELoss()
al_w = 1
generative_loss = nn.MSELoss()
gl_w = 1
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=betas)
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=betas)
optimizer_T1 = torch.optim.Adam(t1_model.parameters(), lr=lr*10, betas=betas)
optimizer_T2 = torch.optim.Adam(t2_model.parameters(), lr=lr*10, betas=betas)

In [86]:
## Change to True / False if you do / do not want 
## to load the previously saved state dictionaries
load = False
if load:
    generator.load_state_dict(torch.load("generator.pt"))
    discriminator.load_state_dict(torch.load("discriminator.pt"))
    t1_model.load_state_dict(torch.load("t1_model.pt"))
    t2_model.load_state_dict(torch.load("t2_model.pt"))

img_list = []
G_losses = []
D_losses = []
iters = 0

batch_size = 2
img_shape = (batch_size, 145, 176, 145)

# Train the models
for epoch in range(num_epochs):
    # # Train the discriminator
    # optimizer_D.zero_grad()
    i = 0
    for t1, t2, fa in tqdm(dataloader):
        t1, t2, fa = t1.to(device), t2.to(device), fa.to(device)
        
        # update D: maximize log(D(x)) + log(1 - D(G(z)))
        # train with all-real batch
        discriminator.zero_grad()
        real_images = fa.float()
        real_labels = torch.ones(real_images.shape[0], 1)#.cuda()
        real_predictions = discriminator(real_images)
        real_loss = adversarial_loss(real_predictions, real_labels)
        real_loss.backward()

        # create latent space and update D with fake image
        t1_latent = t1_model(t1.float()) 
        t2_latent = t2_model(t2.float())
        # latent = t1_latent + t2_latent
        latent = torch.concat((t1_latent, t2_latent), 1)
        fake_images = generator(latent)
        fake_images = torch.nn.functional.pad(fake_images, pad=(0, 1, 3, 3, 0, 1), mode='replicate') # kinda sketch 
        fake_labels = torch.zeros(fake_images.shape[0], 1)#.cuda()
        fake_predictions = discriminator(fake_images.detach())
        fake_loss = adversarial_loss(fake_predictions, fake_labels)
        fake_loss.backward()

        discriminator_loss = real_loss + fake_loss
        optimizer_D.step()

        # update G network: maximize log(D(G(z)))
        generator.zero_grad()
        t1_model.zero_grad()
        t2_model.zero_grad()

        fake_images = generator(latent)
        fake_images = torch.nn.functional.pad(fake_images, pad=(0, 1, 3, 3, 0, 1), mode='replicate') # kinda sketch 
        fake_predictions = discriminator(fake_images)
        errG = adversarial_loss(fake_predictions, real_labels)
        errR = generative_loss(fake_images, fa.float())
        generator_loss = errG + errR
        generator_loss.backward()
        optimizer_G.step()     
        optimizer_T1.step()
        optimizer_T2.step()   

        # output training stats
        #if i % 50 == 0:
        G_losses.append(generator_loss.item())
        D_losses.append(discriminator_loss.item())

        # Print the losses
        if i % 100 == 0:
            print("Epoch [%d/%d], Step [%d/%d], Discriminator Loss: %.4f, Generator Loss: %.4f"
                  % (epoch, num_epochs, i, len(dataloader), discriminator_loss.item(), generator_loss.item()))
            torch.save(generator.state_dict(), "generator.pt")
            torch.save(discriminator.state_dict(), "discriminator.pt")
            torch.save(t1_model.state_dict(), "t1_model.pt")
            torch.save(t2_model.state_dict(), "t2_model.pt")
        i += 1
            
        iters += 1

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch [0/1], Step [0/2], Discriminator Loss: 1.3998, Generator Loss: 39.4128


 50%|█████     | 1/2 [00:21<00:21, 21.76s/it]


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 936537600 bytes.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
load = False
if load:
    generator.load_state_dict(torch.load("generator.pt"))
    discriminator.load_state_dict(torch.load("discriminator.pt"))
    t1_model.load_state_dict(torch.load("t1_model.pt"))
    t2_model.load_state_dict(torch.load("t2_model.pt"))

def generate_img(t1w_image, t2w_image):
    t1w_image = np.stack([t1w_image], axis=0)
    t2w_image = np.stack([t2w_image], axis=0)
    t1_latent = t1_model(t1w_image.float()) 
    t2_latent = t2_model(t2w_image.float())
    latent = torch.concat((t1_latent, t2_latent), 1)
    generated_image = generator(latent)
    return generated_image

In [ ]:
import SimpleITK as sitk

# Change the working directory to the "data" folder
os.chdir('../data/output')

# Get the list of patient folders
patient_folders = [folder for folder in os.listdir() if os.path.isdir(folder)]

# Register images to T1w space using Diffeomorphic Demons
for patient_folder in patient_folders:
    output_folder = os.path.abspath(patient_folder)

    # Create the output folder for the registered images
    synthesized_output_folder = os.path.join(output_folder, "synthesized")
    os.makedirs(synthesized_output_folder, exist_ok=True)

    t1w_image_path = os.path.join(output_folder, "registered", "T1w_1mm_normalized.nii.gz")
    t2w_image_path = os.path.join(output_folder, "registered", "T2w_1mm_normalized.nii.gz")
    fa_image_path = os.path.join(output_folder, "registered", "FA_1.25mm_normalized.nii.gz")
    adc_image_path = os.path.join(output_folder, "registered", "ADC_1.25mm_normalized.nii.gz")

    # synthesized_image_path = os.path.join(synthesized_output_folder, "ADC_synthesized.nii.gz")
    synthesized_image_path = os.path.join(synthesized_output_folder, "FA_synthesized.nii.gz")

    t1w_image_file = nib.load(t1w_image_path)
    t1w_image = t1w_image_file.get_fdata()
    t2w_image_file = nib.load(t2w_image_path)
    t2w_image = t2w_image_file.get_fdata()
    fa_image_file = nib.load(fa_image_path)
    fa_image = fa_image_file.get_fdata()
    adc_image_file = nib.load(adc_image_path)
    adc_image = adc_image_file.get_fdata()

    generated_image = generate_img(t1w_image, t2w_image)

    # visualize output vs actual file
    plt.rcParams["figure.figsize"]=20,20
    plt.figure()
    plt.subplot(1,4,1)
    plt.imshow(generated_image[:,:,100])
    plt.title('Generated Image')
    plt.subplot(1,4,2)
    plt.imshow(fa_image[:,:,100])
    plt.title('Actual Image')
    plt.subplot(1,4,3)
    plt.imshow(generated_image[105,:,:])
    plt.title('Generated Image')
    plt.subplot(1,4,4)
    plt.imshow(fa_image[105,:,:])
    plt.title('Actual Image')
    plt.show()

    sitk.WriteImage(generated_image, synthesized_image_path)